# Notebook 3: Results Analysis and Publication Figures

Generate all publication-ready figures and tables.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.features import FeatureEngineer
from src.uplift_model import TLearner, ResponseModel
from src.ltv_model import LTVModel
from src.churn_model import ChurnModel
from src.scoring import score_players, ScoringPipeline
from src.evaluation import UpliftEvaluator

%matplotlib inline

## 1. Load Models and Data

In [ ]:
# Load test data
test_df = pd.read_csv('../data/player_data_test.csv')

# Load models
uplift_model = TLearner.load('../data/uplift_model.pkl')
response_model = ResponseModel.load('../data/response_model.pkl')
ltv_model = LTVModel.load('../data/ltv_model.pkl')
churn_model = ChurnModel.load('../data/churn_model.pkl')

# Prepare features
fe = FeatureEngineer(scale_features=True)
train_df = pd.read_csv('../data/player_data_train.csv')
fe.prepare_features(train_df, fit_scaler=True)

print(f"Test samples: {len(test_df)}")

## 2. Generate All Predictions

In [ ]:
# Score all players
test_df = score_players(test_df, uplift_model, ltv_model, churn_model, fe)

# Add response model predictions
X_test = fe.prepare_features(test_df, fit_scaler=False)
test_df['response_score'] = response_model.predict_response(X_test)

print("Predictions generated")
test_df[['player_id', 'segment', 'uplift_score', 'response_score', 'composite_score']].head()

## 3. Model Comparison Metrics

In [ ]:
evaluator = UpliftEvaluator()

comparison_df = evaluator.compare_models(
    test_df,
    models={
        'Uplift Model': 'uplift_score',
        'Response Model': 'response_score',
        'True Uplift (Oracle)': 'true_uplift'
    },
    budget_constraint=0.3
)

print("=== MODEL COMPARISON (30% Targeting Budget) ===")
print(comparison_df.to_string(index=False))

# Save table
comparison_df.to_csv('../results/tables/model_comparison.csv', index=False)

## 4. Figure 1: Segment Distribution

In [ ]:
fig = evaluator.plot_segment_distribution(
    test_df,
    save_path='../results/figures/figure1_segment_distribution.png'
)
plt.show()

## 5. Figure 2: Uplift Calibration

In [ ]:
fig = evaluator.plot_uplift_calibration(
    test_df,
    n_bins=10,
    save_path='../results/figures/figure2_uplift_calibration.png'
)
plt.show()

## 6. Figure 3: Cumulative Gains (Qini Curves)

In [ ]:
fig = evaluator.plot_cumulative_gains(
    test_df,
    models={
        'Uplift Model': 'uplift_score',
        'Response Model': 'response_score',
        'True Uplift (Oracle)': 'true_uplift'
    },
    save_path='../results/figures/figure3_cumulative_gains.png'
)
plt.show()

## 7. Segment Analysis Table

In [ ]:
segment_analysis = test_df.groupby('segment').agg({
    'player_id': 'count',
    'true_uplift': 'mean',
    'uplift_score': 'mean',
    'ltv_score': 'mean',
    'churn_probability': 'mean',
    'selected': 'sum'
}).round(3)

segment_analysis.columns = ['n_players', 'true_uplift', 'pred_uplift', 'avg_ltv', 'avg_churn', 'n_selected']
segment_analysis = segment_analysis.reset_index()

print("\n=== SEGMENT ANALYSIS ===")
print(segment_analysis.to_string(index=False))

segment_analysis.to_csv('../results/tables/segment_analysis.csv', index=False)

## 8. Key Statistics for Paper

In [ ]:
uplift_improvement = comparison_df[comparison_df['model'] == 'Uplift Model']['improvement_vs_random'].values[0]
response_improvement = comparison_df[comparison_df['model'] == 'Response Model']['improvement_vs_random'].values[0]

print("\n=== KEY FINDINGS FOR PAPER ===")
print(f"\n1. Uplift model achieved {uplift_improvement:.1f}% improvement over random targeting")
print(f"2. Response model achieved {response_improvement:.1f}% improvement over random targeting")
print(f"3. Uplift model outperformed response model by {uplift_improvement - response_improvement:.1f} percentage points")

print("\n4. Player Segments:")
for segment in ['persuadables', 'sure_things', 'lost_causes', 'sleeping_dogs']:
    pct = (test_df['segment'] == segment).sum() / len(test_df) * 100
    avg_uplift = test_df[test_df['segment'] == segment]['true_uplift'].mean()
    print(f"   - {segment.replace('_', ' ').title()}: {pct:.1f}% (avg uplift: {avg_uplift:+.3f})")

print("\n✓ Analysis complete! All figures and tables saved to results/")